In [9]:
from dataset import cargarDataset
from pathlib import Path
import time
from ej2 import pinvEcuacionesNormales
from ej3 import pinvSVD
from ej4 import pinvHouseHolder, pinvGramSchmidt
from ej5 import esPseudoInversa
from ej6 import validate_transferlearning, matriz_confusion
from alc import svd_reducida, QR_con_GS

data_path = Path("./dataset/cats_and_dogs")
X_train, Y_train, X_val, Y_val = cargarDataset(data_path)

In [10]:
    #Cholesky
    print("running cholesky...")
    start_time = time.perf_counter()
    W = pinvEcuacionesNormales(X_train, Y_train)
    end_time = time.perf_counter()
    Cholesky_time = end_time - start_time
    print(f"Cholesky exercise executed in: {Cholesky_time:.4f} seconds")
    Cholesky_accuracy = validate_transferlearning(W,X_val,Y_val)
    #matriz_confusion(W, X_val, Y_val)

running cholesky...
Rango de X: 1536
Cholesky exercise executed in: 111.9539 seconds

--- Resultados de Validación ---
Precisión (Accuracy): 12.80%
Clasificó correctamente 128 de 1000 muestras.


In [ ]:
    #SVD
    print("running svd...")
    start_time = time.perf_counter()
    U, S, V = svd_reducida(X_train)
    W_SVD = pinvSVD(U, S, V, Y_train)
    end_time = time.perf_counter()
    SVD_time = end_time - start_time
    print(f"SVD exercise executed in: {SVD_time:.4f} seconds")
    SVD_accuracy = validate_transferlearning(W_SVD,X_val,Y_val)
    matriz_confusion(W_SVD, X_val, Y_val)


running svd...
Calculando A*AT...


In [ ]:
    #QR
    print("running qr gs...")
    start_time = time.perf_counter()
    Q, R = QR_con_GS(X_train)
    W_GS =  pinvGramSchmidt(Q, R, Y_train)
    end_time = time.perf_counter()
    GS_time = end_time - start_time
    print(f"GS exercise executed in: {GS_time:.4f} seconds")
    GS_accuracy = validate_transferlearning(W_GS,X_val,Y_val)
    matriz_confusion(W_GS, X_val, Y_val)

In [ ]:
    print("\n" + "--- Tablas de resultados ---".center(45))
    print(" " * 15 + " Tiempo | Accuracy |")
    print(" " * 17 + "-" * 37)
    print(f"SVD  | {SVD_time:^15} | {SVD_accuracy:^17} |")
    print(" " * 17 + "-" * 37)
    print(f"GS | {GS_time:^15} | {GS_accuracy:^17} |")
    print(" " * 17 + "-" * 37)
    print(f"CHL | {Cholesky_time:^15} | {Cholesky_accuracy:^17} |")
    print(" " * 17 + "-" * 37)
    print("\n")